In [0]:
from sklearn.feature_selection import RFECV, SelectKBest, SelectPercentile, f_classif
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from google.colab import files
from sklearn.svm import SVC
import seaborn as sns
import pandas as pd
import numpy as np

In [0]:
import warnings
warnings.filterwarnings("ignore")

# **Import**

Import training set1, training set2 and test set

In [0]:
url1 = '.../Class_labels_He.txt'
train1 = pd.read_csv(url1, header=0, delimiter='\t')
url2 = '.../TrainingSchirmer_TaxonomyAbundance_matrix.txt'
train2 = pd.read_csv(url2, header=0, delimiter='\t')
url3 = '.../TestingDataset_TaxonomyAbundance_matrix.txt'
test = pd.read_csv(url3, header=0, delimiter='\t')

Check dimension of data

In [0]:
print('Number of samples')
print('- taxonomy abudance matrix (training set1): ',train1.shape[1]-1)
print('- taxonomy abudance matrix (training set2): ',train2.shape[1]-1)

In [0]:
print('Number of features') 
print('- taxonomy abudance matrix (training set1): ',train1.shape[0])
print('- taxonomy abudance matrix (training set2): ',train2.shape[0])

Import class labels for training set1 and class labels for training set2

In [0]:
url4 = '.../Class_labels_He.txt'
class_labels_He = pd.read_csv(url4, header=0, delimiter='\t')
url5 = '.../Class_labels_Schirmer.txt'
class_labels_Sch = pd.read_csv(url5, header=0, delimiter='\t')

# **Data processing**

Transform datasets in suitable format

In [0]:
def normalize(df):
  for i in range(df.shape[0]):
    sum_row = df.iloc[i].sum()
    df.iloc[i] = df.iloc[i].apply(lambda x: x/sum_row)
  return df

def extractX(df):
    """Df - dataframe: rows=features; columns=samples"""

    X = df.iloc[:,1:]
    X = X.T
    X = normalize(X)
    
    return X

In [0]:
X1 = extractX(train1)
X2 = extractX(train2)
new_test = extractX(test)
y1 = class_labels_He.iloc[:,2:]
y2 = class_labels_Sch.iloc[:,2:]

In [0]:
X = pd.concat([X1,X2],axis=0)
y = pd.concat([y1,y2],axis=0)
y = y.to_numpy()
y = y.reshape(len(y))

In [0]:
def int_encoder(y, case):
    if case == 4:
      cat_to_int = {'UC':0, 'CD': 1}
    else:
      cat_to_int = {'UC': 1,'CD': 1, 'nonIBD': 0}
    y = [cat_to_int[i] for i in y]
    return np.array(y)

def train_test(data, labels, case):
  merged = data.copy()
  merged['group'] = labels
  if case == 2:
    merged = merged[merged['group'] != 'CD']
  if case == 3:
    merged = merged[merged['group'] != 'UC']
  if case == 4:
    merged = merged[merged['group'] != 'nonIBD']
  
  X3, y3 = merged.iloc[:,:-1], merged.iloc[:,-1]
  X3 = X3.to_numpy()
  y3 = int_encoder(y3, case)
  return X3, y3

# **Classification and Feature Selection**

In [0]:
min_features = 5
removal_step = 25

In [0]:
def classification(estimator, p, model, data, labels):
  pipeline = 0
  if model == 'skb':
    skb = SelectKBest(f_classif)
    pipeline = Pipeline([('skb', skb),('estimator', estimator)])
    params = dict(p.items())
    params['skb__k'] = range(500, min_features, -removal_step)
  elif model == 'skp':
    skp = SelectPercentile(f_classif)
    pipeline = Pipeline([('skp', skp),('estimator', estimator)])
    params = dict(p.items())
    params['skp__percentile'] = [0.1,0.3,0.5,0.7,0.9]
  elif model=='rfecv':
    params = dict(p.items())
    pipeline = RFECV(estimator, step=0.7, min_features_to_select=min_features, scoring='f1',cv=10, n_jobs=-1)
  
  gs = GridSearchCV(pipeline, param_grid=params, cv=10, n_jobs=-1, scoring="f1", refit=True)
  gs.fit(data, labels)
  best_params = gs.best_params_
  best_model = gs.best_estimator_
  return best_model, best_params

Random Forest

In [0]:
rf = RandomForestClassifier()
params_rf = {
    "estimator__n_estimators": [100,150,200,300],
    "estimator__max_depth": [None],
    "estimator__max_features": ["sqrt"],
    "estimator__min_samples_split": [2,3],
    "estimator__min_samples_leaf": [2,3,4],
    "estimator__class_weight": ['balanced'],
    "estimator__bootstrap": [False, True],
    "estimator__random_state" : [42]
}

SVC

In [0]:
svc = SVC(probability=True, random_state=0)
params_svc = {
    "estimator__C": [10, 50, 100, 200,1000], 
    "estimator__gamma": [ 0.15, 0.2, 0.25, 0.3, 0.4],
    "estimator__kernel": ['linear','poly','rbf'],
    "estimator__class_weight": ['balanced'],
    "estimator__degree": [3,4,5]
    }

## **C1: IBD vs nonIBD**

In [0]:
train_C1, y_C1 = train_test(X, y, case=1)

In [0]:
bm_rfSK_C1, bp_rfSK_C1 = classification(rf, params_rf, 'skb', train_C1, y_C1)
bm_rfSP_C1, bp_rfSP_C1 = classification(rf, params_rf, 'skp', train_C1, y_C1)
bm_rfrfecv_C1, bp_rfrfecv_C1= classification(rf, params_rf, 'rfecv', train_C1, y_C1)

In [0]:
bm_svcSK_C1, bp_svcSK_C1 = classification(svc, params_svc, 'skb', train_C1, y_C1)
bm_svcSP_C1, bp_svcSP_C1 = classification(svc, params_svc, 'skp', train_C1, y_C1)

In [0]:
best_params_C1 = [bp_rfSK_C1, bp_rfSP_C1, bp_rfrfecv_C1, bp_svcSK_C1, bp_svcSP_C1]

## **C2:UC vs nonIBD**

In [0]:
train_C2, y_C2 = train_test(X, y, case=2)

Random Forest

In [0]:
bm_rfSK_C2, bp_rfSK_C2 = classification(rf, params_rf, 'skb', train_C2, y_C2)
bm_rfSP_C2, bp_rfSP_C2 = classification(rf, params_rf, 'skp', train_C2, y_C2)
bm_rfrfecv_C2, bp_rfrfecv_C2 = classification(rf, params_rf, 'rfecv', train_C2, y_C2)

SVC

In [0]:
bm_svcSK_C2, bp_svcSK_C2 = classification(svc, params_svc, 'skb', train_C2, y_C2)
bm_svcSP_C2, bp_svcSP_C2 = classification(svc, params_svc, 'skp', train_C2, y_C2)

In [0]:
best_params_C2 = [bp_rfSK_C2, bp_rfSP_C2, bp_rfrfecv_C2, bp_svcSK_C2, bp_svcSP_C2]

## **C3:UC vs CD**

In [0]:
train_C3, y_C3 = train_test(X, y, case=3)

Random Forest

In [0]:
bm_rfSK_C3, bp_rfSK_C3 = classification(rf, params_rf, 'skb', train_C3, y_C3)
bm_rfSP_C3, bp_rfSP_C3 = classification(rf, params_rf, 'skp', train_C3, y_C3)
bm_rfrfecv_C3, bp_rfrfecv_C3 = classification(rf, params_rf, 'rfecv', train_C3, y_C3)

SVM

In [0]:
bm_svcSK_C3, bp_svcSK_C3 = classification(svc, params_svc, 'skb', train_C3, y_C3)
bm_svcSP_C3, bp_svcSP_C3 = classification(svc, params_svc, 'skp', train_C3, y_C3)

In [0]:
best_params_C3 = [bp_rfSK_C3, bp_rfSP_C3, bp_rfrfecv_C3, bp_svcSK_C3, bp_svcSP_C3]

## **C4:UC vs CD**

In [0]:
train_C4, y_C4 = train_test(X, y, case=4)

Random Forest

In [0]:
bm_rfSK_C4, bp_rfSK_C4 = classification(rf, params_rf, 'skb', train_C4, y_C4)
bm_rfSP_C4, bp_rfSP_C4 = classification(rf, params_rf, 'skp', train_C4, y_C4)
bm_rfrfecv_C4, bp_rfrfecv_C4 = classification(rf, params_rf, 'rfecv', train_C4, y_C4)

SVM

In [0]:
bm_svcSK_C4, bp_svcSK_C4 = classification(svc, params_svc, 'skb', train_C4, y_tr_C4)
bm_svcSP_C4, bp_svcSP_C4 = classification(svc, params_svc, 'skp', train_C4, y_tr_C4)

In [0]:
best_params_C4 = [bp_rfSK_C4, bp_rfSP_C4, bp_rfrfecv_C4, bp_svcSK_C4, bp_svcSP_C4]

# **Perfomance**

In [0]:
def performance(models, models_names, data, labels, classes):
  f1 = []
  acc = []
  for i, model in enumerate(models):
    y_pred = np.empty(labels.shape, dtype='int')
    kf = StratifiedKFold(n_splits=10)
    for train, test in kf.split(data, labels):
      model.fit(data[train], labels[train])
      y_pred[test] = model.predict(data[test])
    
    f1.append(f1_score(labels, y_pred))
    acc.append(accuracy_score(labels, y_pred))
    cm = confusion_matrix(labels, y_pred)
    df_cm = pd.DataFrame(cm, classes, classes)
    sns.set(font_scale=1.4) # for label size
    sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 16})
    plt.title('Confusion matrix ' + models_names[i])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

  data = {'Models': models_names,'Accuracy': acc, 'F1': f1}
  df = pd.DataFrame(data)
  
  return df

In [0]:
names = ['SK+rf','SP+rf','RFECV+rf', 'SK+svc','SP+svc']



## C1



In [0]:
classes1 = ['nonIBD', 'IBD']
best_models1 = [bm_rfSK_C1, bm_rfSP_C1, bm_rfrfecv_C1, bm_svcSK_C1, bm_svcSP_C1]
scores1 = performance(best_models1, names, train_C1, y_C1, classes1)

## C2

In [0]:
classes2 = ['nonIBD', 'UC']
best_models2 = [bm_rfSK_C2, bm_rfSP_C2, bm_rfrfecv_C2, bm_svcSK_C2, bm_svcSP_C2]
scores2 = performance(best_models2, names, train_C2, y_C2, classes2)

## C3

In [0]:
classes3 = ['nonIBD', 'CD']
best_models3 = [bm_rfSK_C3, bm_rfSP_C3, bm_rfrfecv_C3, bm_svcSK_C3, bm_svcSP_C3]
scores3 = performance(best_models3, names, train_C3, y_C3, classes3)

## C4

In [0]:
classes = ['UC', 'CD']
best_models4 = [bm_rfSK_C4, bm_rfSP_C4, bm_rfrfecv_C4, bm_svcSK_C4, bm_svcSP_C4]
scores4 = performance(best_models4, names, train_C4, y_C4)